### evs.ipynb
This notebook is for manipulating and working with the ESS8 data!

- eusclbf ranges between 1 (strongly against) and 4 (strongly in favour) (anything above this is refusal/dk/na)
- ipeqopt ranges between 1 (very much like me) and 6 (not like me at all) with 7-9 being refusal/dk/na
- ipgdtim ranges between 1 (very much like me) and 6 (not like me at all) with 7-9 being refusal/dk/na

In [67]:
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [68]:
filename = '/home/ia23938/Downloads/ESS8e02_3-subset.csv'

df = pd.read_csv(filename)
df = df.drop(columns=['name', 'essround', 'edition', 'proddate', 'idno', 'dweight', 'pspwght', 'pweight', 'anweight'])
df = df.dropna()

action = "eusclbf"
value1 = "ipeqopt"
value2 = "ipgdtim"

# remove weighting from N/A, DK, and REF values
df.loc[df['ipeqopt'] > 6, 'ipeqopt'] = pd.NA
df.loc[df['ipgdtim'] > 6, 'ipgdtim'] = pd.NA
df.loc[df['eusclbf'] > 4, 'eusclbf'] = pd.NA


columns = {'cntry':[], 'P[ipeqopt, ipgdtim]':[], 'P[ipgdtim, ipeqopt]':[], 
           'a_ipeqopt(for eusclbf)':[], 'a_ipgdtim(for eusclbf)':[], 
           'a_ipeqopt(against eusclbf)':[], 'a_ipgdtim(against eusclbf)':[]}
value_system_df = pd.DataFrame(columns=columns)

for country in df['cntry'].unique():
    # Value preferences
    country_df = df[df['cntry'] == country].copy()
    country_df['preferred_action'] = country_df[value1] < country_df[value2]

    country_df['a_value1_action'] = country_df.apply(lambda row: calculate_action_value(row, value1), axis=1)
    country_df['a_value2_action'] = country_df.apply(lambda row: calculate_action_value(row, value2), axis=1)

    # finally, add to dataframe
    to_append = {'cntry': country, 'P[ipeqopt, ipgdtim]': country_df['preferred_action'].mean(), 'P[ipgdtim, ipeqopt]': 1-country_df['preferred_action'].mean(),
                 'a_ipeqopt(for eusclbf)': country_df['a_value1_action'].mean(), 'a_ipgdtim(for eusclbf)': country_df['a_value2_action'].mean(),
                 'a_ipeqopt(against eusclbf)': -1*country_df['a_value1_action'].mean(), 'a_ipgdtim(against eusclbf)': -1*country_df['a_value2_action'].mean()
                 }
    value_system_df = value_system_df._append(to_append, ignore_index=True)

value_system_df


/tmp/ipykernel_56934/1142978716.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  value_system_df = value_system_df._append(to_append, ignore_index=True)


,cntry,"P[ipeqopt, ipgdtim]","P[ipgdtim, ipeqopt]",a_ipeqopt(for eusclbf),a_ipgdtim(for eusclbf),a_ipeqopt(against eusclbf),a_ipgdtim(against eusclbf)
0,AT,0.456219,0.543781,-0.518392,-0.348921,0.518392,0.348921
1,BE,0.484711,0.515289,-0.580227,-0.351622,0.580227,0.351622
2,CZ,0.430586,0.569414,-0.354079,-0.205843,0.354079,0.205843
3,DE,0.431627,0.568373,-0.567751,-0.426996,0.567751,0.426996
4,EE,0.386330,0.613670,-0.339373,-0.277689,0.339373,0.277689
5,ES,0.662411,0.337589,-0.736223,-0.170531,0.736223,0.170531
6,FI,0.586494,0.413506,-0.563885,-0.203577,0.563885,0.203577
7,FR,0.423671,0.576329,-0.616521,-0.437476,0.616521,0.437476
8,GB,0.664114,0.335886,-0.534122,-0.003296,0.534122,0.003296
9,HU,0.324040,0.675960,-0.504658,-0.439506,0.504658,0.439506
